# Imagenet Training MobileNet


In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
# Just some functions to average stuff, and save the model
from utils_pytorch import *
from sutils import AverageMeter,accuracy

# Trainning parameters
learning_rate = 0.1
batch_size = 64
momentum = 0.9
weight_decay = 1e-4
workers = 1
print_freq = 100
epochs = 1000

# Define your data path here
IMAGENET_PATH = './dataset'

### Define Mobilenet class
#### Architecture
![title](ArchMobileNet.png)
#### Normal Convolution and Depthwise convolution
![title](MobileNetConvs.png)

In [2]:
class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()

        # Normal convolution block followed by Batchnorm (CONV_3x3-->BN-->Relu)
        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        # Depthwise convolution block (CONV_BLK_3x3-->BN-->Relu-->CONV_1x1-->BN-->Relu)
        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 2), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7),
        )
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

### Initialize model and pass to the GPU

In [3]:
model = MobileNet()

#print(model)

# use GPU to accelerate
model = torch.nn.DataParallel(model).cuda()

### Define Loss

In [4]:
criterion = nn.CrossEntropyLoss().cuda()

### Define solver(SGD)

In [5]:
optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                                momentum=momentum,
                                weight_decay=weight_decay)

### Data loading specifics for ImageNet

In [6]:
# Data loading code
traindir = os.path.join(IMAGENET_PATH, 'train')
valdir = os.path.join(IMAGENET_PATH, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])

# Preprocess the data, Operations that will be done on data
train_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(traindir, transforms.Compose([
            transforms.RandomResizedCrop(224), #randomly crop image to 224 * 224
            transforms.RandomHorizontalFlip(), 
            transforms.RandomVerticalFlip(), 
#             transforms.RandomRotation(10), 
            transforms.ColorJitter(0.05, 0.05, 0.05, 0.05), 
            transforms.ToTensor(), 
            normalize,
        ])),
        batch_size=batch_size, shuffle=True,
        num_workers=workers, pin_memory=True)

print(train_loader)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=workers, pin_memory=True)
print(val_loader)

### Train

In [7]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        #losses.update(loss.data[0], input.size(0))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))
            
            
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))

    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top1.avg

In [8]:
best_prec1 = 0
for epoch in range(0, epochs):
        adjust_learning_rate(optimizer, epoch, learning_rate)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        prec1 = validate(val_loader,model, criterion)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer' : optimizer.state_dict(),
        }, is_best)

Epoch: [0][0/60]	Time 0.860 (0.860)	Data 0.783 (0.783)	Loss 6.8219 (6.8219)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test: [0/27]	Time 0.457 (0.457)	Loss 0.2767 (0.2767)	Prec@1 93.750 (93.750)	Prec@5 93.750 (93.750)
 * Prec@1 49.070 Prec@5 74.085
Epoch: [1][0/60]	Time 0.595 (0.595)	Data 0.556 (0.556)	Loss 2.7096 (2.7096)	Prec@1 25.000 (25.000)	Prec@5 54.688 (54.688)
Test: [0/27]	Time 0.393 (0.393)	Loss 0.4284 (0.4284)	Prec@1 93.750 (93.750)	Prec@5 93.750 (93.750)
 * Prec@1 53.329 Prec@5 86.203
Epoch: [2][0/60]	Time 0.585 (0.585)	Data 0.547 (0.547)	Loss 2.0455 (2.0455)	Prec@1 46.875 (46.875)	Prec@5 71.875 (71.875)
Test: [0/27]	Time 0.371 (0.371)	Loss 0.8448 (0.8448)	Prec@1 81.250 (81.250)	Prec@5 95.312 (95.312)
 * Prec@1 60.108 Prec@5 86.263
Epoch: [3][0/60]	Time 0.579 (0.579)	Data 0.540 (0.540)	Loss 1.7383 (1.7383)	Prec@1 51.562 (51.562)	Prec@5 82.812 (82.812)
Test: [0/27]	Time 0.380 (0.380)	Loss 0.2705 (0.2705)	Prec@1 93.750 (93.750)	Prec@5 96.875 (96.875)
 * Prec@1 73.905 Prec@5 93.221
Epoch: [4][0/60]	Time 0.596 (0.596)	Data 0.557 (0.557)	Loss 1.5734 (1.5734)	Prec@1 62.500 (62.500)	Prec@5 85.938 (

Epoch: [33][0/60]	Time 0.573 (0.573)	Data 0.534 (0.534)	Loss 0.5307 (0.5307)	Prec@1 84.375 (84.375)	Prec@5 95.312 (95.312)
Test: [0/27]	Time 0.369 (0.369)	Loss 0.3520 (0.3520)	Prec@1 90.625 (90.625)	Prec@5 98.438 (98.438)
 * Prec@1 94.841 Prec@5 98.920
Epoch: [34][0/60]	Time 0.581 (0.581)	Data 0.543 (0.543)	Loss 0.7626 (0.7626)	Prec@1 76.562 (76.562)	Prec@5 93.750 (93.750)
Test: [0/27]	Time 0.381 (0.381)	Loss 0.3420 (0.3420)	Prec@1 92.188 (92.188)	Prec@5 98.438 (98.438)
 * Prec@1 94.901 Prec@5 98.860
Epoch: [35][0/60]	Time 0.579 (0.579)	Data 0.540 (0.540)	Loss 0.7398 (0.7398)	Prec@1 82.812 (82.812)	Prec@5 92.188 (92.188)
Test: [0/27]	Time 0.371 (0.371)	Loss 0.2793 (0.2793)	Prec@1 92.188 (92.188)	Prec@5 98.438 (98.438)
 * Prec@1 94.721 Prec@5 98.920
Epoch: [36][0/60]	Time 0.591 (0.591)	Data 0.553 (0.553)	Loss 0.6488 (0.6488)	Prec@1 79.688 (79.688)	Prec@5 95.312 (95.312)
Test: [0/27]	Time 0.368 (0.368)	Loss 0.2905 (0.2905)	Prec@1 92.188 (92.188)	Prec@5 98.438 (98.438)
 * Prec@1 94.961 Pr

Test: [0/27]	Time 0.373 (0.373)	Loss 0.2926 (0.2926)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.441 Prec@5 99.160
Epoch: [66][0/60]	Time 0.581 (0.581)	Data 0.542 (0.542)	Loss 0.3454 (0.3454)	Prec@1 92.188 (92.188)	Prec@5 100.000 (100.000)
Test: [0/27]	Time 0.374 (0.374)	Loss 0.2828 (0.2828)	Prec@1 92.188 (92.188)	Prec@5 98.438 (98.438)
 * Prec@1 95.321 Prec@5 98.980
Epoch: [67][0/60]	Time 0.584 (0.584)	Data 0.544 (0.544)	Loss 0.3303 (0.3303)	Prec@1 89.062 (89.062)	Prec@5 100.000 (100.000)
Test: [0/27]	Time 0.372 (0.372)	Loss 0.3050 (0.3050)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.441 Prec@5 99.100
Epoch: [68][0/60]	Time 0.599 (0.599)	Data 0.559 (0.559)	Loss 0.6113 (0.6113)	Prec@1 87.500 (87.500)	Prec@5 95.312 (95.312)
Test: [0/27]	Time 0.383 (0.383)	Loss 0.2844 (0.2844)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.441 Prec@5 99.160
Epoch: [69][0/60]	Time 0.586 (0.586)	Data 0.546 (0.546)	Loss 0.4157 (0.4157)	Prec@1 87.500 (87.500)	Prec@5 

...

Test: [0/27]	Time 0.387 (0.387)	Loss 0.2739 (0.2739)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.321 Prec@5 99.100
Epoch: [846][0/60]	Time 0.620 (0.620)	Data 0.576 (0.576)	Loss 0.3949 (0.3949)	Prec@1 87.500 (87.500)	Prec@5 98.438 (98.438)
Test: [0/27]	Time 0.410 (0.410)	Loss 0.3056 (0.3056)	Prec@1 92.188 (92.188)	Prec@5 98.438 (98.438)
 * Prec@1 95.381 Prec@5 99.040
Epoch: [847][0/60]	Time 0.609 (0.609)	Data 0.566 (0.566)	Loss 0.4252 (0.4252)	Prec@1 90.625 (90.625)	Prec@5 100.000 (100.000)
Test: [0/27]	Time 0.403 (0.403)	Loss 0.3011 (0.3011)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.381 Prec@5 99.100
Epoch: [848][0/60]	Time 0.603 (0.603)	Data 0.565 (0.565)	Loss 0.3675 (0.3675)	Prec@1 90.625 (90.625)	Prec@5 98.438 (98.438)
Test: [0/27]	Time 0.384 (0.384)	Loss 0.2840 (0.2840)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.441 Prec@5 99.040
Epoch: [849][0/60]	Time 0.617 (0.617)	Data 0.571 (0.571)	Loss 0.5044 (0.5044)	Prec@1 84.375 (84.375)	Prec@

 * Prec@1 95.381 Prec@5 99.040
Epoch: [878][0/60]	Time 0.617 (0.617)	Data 0.574 (0.574)	Loss 0.5799 (0.5799)	Prec@1 82.812 (82.812)	Prec@5 93.750 (93.750)
Test: [0/27]	Time 0.389 (0.389)	Loss 0.2977 (0.2977)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.441 Prec@5 99.040
Epoch: [879][0/60]	Time 0.592 (0.592)	Data 0.554 (0.554)	Loss 0.4442 (0.4442)	Prec@1 84.375 (84.375)	Prec@5 98.438 (98.438)
Test: [0/27]	Time 0.417 (0.417)	Loss 0.2879 (0.2879)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.381 Prec@5 99.100
Epoch: [880][0/60]	Time 0.662 (0.662)	Data 0.622 (0.622)	Loss 0.4060 (0.4060)	Prec@1 87.500 (87.500)	Prec@5 98.438 (98.438)
Test: [0/27]	Time 0.406 (0.406)	Loss 0.2867 (0.2867)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.261 Prec@5 99.100
Epoch: [881][0/60]	Time 0.622 (0.622)	Data 0.574 (0.574)	Loss 0.4555 (0.4555)	Prec@1 87.500 (87.500)	Prec@5 98.438 (98.438)
Test: [0/27]	Time 0.414 (0.414)	Loss 0.2893 (0.2893)	Prec@1 93.750 (93.750)	Prec@5 

Test: [0/27]	Time 0.432 (0.432)	Loss 0.2801 (0.2801)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.321 Prec@5 99.100
Epoch: [911][0/60]	Time 0.691 (0.691)	Data 0.646 (0.646)	Loss 0.4417 (0.4417)	Prec@1 89.062 (89.062)	Prec@5 96.875 (96.875)
Test: [0/27]	Time 0.414 (0.414)	Loss 0.2916 (0.2916)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.381 Prec@5 99.160
Epoch: [912][0/60]	Time 0.632 (0.632)	Data 0.586 (0.586)	Loss 0.4391 (0.4391)	Prec@1 85.938 (85.938)	Prec@5 96.875 (96.875)
Test: [0/27]	Time 0.384 (0.384)	Loss 0.2860 (0.2860)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.381 Prec@5 99.100
Epoch: [913][0/60]	Time 0.585 (0.585)	Data 0.547 (0.547)	Loss 0.2849 (0.2849)	Prec@1 87.500 (87.500)	Prec@5 100.000 (100.000)
Test: [0/27]	Time 0.381 (0.381)	Loss 0.3135 (0.3135)	Prec@1 92.188 (92.188)	Prec@5 98.438 (98.438)
 * Prec@1 95.141 Prec@5 99.100
Epoch: [914][0/60]	Time 0.589 (0.589)	Data 0.550 (0.550)	Loss 0.4297 (0.4297)	Prec@1 84.375 (84.375)	Prec@

 * Prec@1 95.381 Prec@5 99.160
Epoch: [943][0/60]	Time 0.637 (0.637)	Data 0.596 (0.596)	Loss 0.4188 (0.4188)	Prec@1 89.062 (89.062)	Prec@5 98.438 (98.438)
Test: [0/27]	Time 0.441 (0.441)	Loss 0.2874 (0.2874)	Prec@1 92.188 (92.188)	Prec@5 98.438 (98.438)
 * Prec@1 95.261 Prec@5 99.040
Epoch: [944][0/60]	Time 0.589 (0.589)	Data 0.551 (0.551)	Loss 0.5713 (0.5713)	Prec@1 79.688 (79.688)	Prec@5 96.875 (96.875)
Test: [0/27]	Time 0.413 (0.413)	Loss 0.2915 (0.2915)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.321 Prec@5 99.100
Epoch: [945][0/60]	Time 0.666 (0.666)	Data 0.620 (0.620)	Loss 0.5141 (0.5141)	Prec@1 85.938 (85.938)	Prec@5 93.750 (93.750)
Test: [0/27]	Time 0.389 (0.389)	Loss 0.2668 (0.2668)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.321 Prec@5 99.040
Epoch: [946][0/60]	Time 0.645 (0.645)	Data 0.606 (0.606)	Loss 0.3905 (0.3905)	Prec@1 87.500 (87.500)	Prec@5 95.312 (95.312)
Test: [0/27]	Time 0.386 (0.386)	Loss 0.2867 (0.2867)	Prec@1 93.750 (93.750)	Prec@5 

Test: [0/27]	Time 0.386 (0.386)	Loss 0.2670 (0.2670)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.501 Prec@5 99.040
Epoch: [976][0/60]	Time 0.587 (0.587)	Data 0.547 (0.547)	Loss 0.6261 (0.6261)	Prec@1 81.250 (81.250)	Prec@5 95.312 (95.312)
Test: [0/27]	Time 0.507 (0.507)	Loss 0.2750 (0.2750)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.441 Prec@5 99.040
Epoch: [977][0/60]	Time 0.846 (0.846)	Data 0.789 (0.789)	Loss 0.5295 (0.5295)	Prec@1 81.250 (81.250)	Prec@5 96.875 (96.875)
Test: [0/27]	Time 0.570 (0.570)	Loss 0.2897 (0.2897)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.441 Prec@5 99.100
Epoch: [978][0/60]	Time 0.912 (0.912)	Data 0.863 (0.863)	Loss 0.5090 (0.5090)	Prec@1 89.062 (89.062)	Prec@5 95.312 (95.312)
Test: [0/27]	Time 0.450 (0.450)	Loss 0.2987 (0.2987)	Prec@1 93.750 (93.750)	Prec@5 98.438 (98.438)
 * Prec@1 95.321 Prec@5 99.160
Epoch: [979][0/60]	Time 0.617 (0.617)	Data 0.573 (0.573)	Loss 0.4909 (0.4909)	Prec@1 90.625 (90.625)	Prec@5 

## Save model

There are two ways to save the model.

**Reference:**

http://pytorch.org/docs/0.3.1/notes/serialization.html#recommend-saving-models

### Only save the parameters (Recommended)

In [9]:
# Save the parameters
torch.save(model.state_dict(),"mobilenet_params_100e.pth.tar")

### Save the whole model